In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from operator import itemgetter
from sklearn.datasets import fetch_mldata

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical

%matplotlib notebook

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.33

In [3]:
### Training Data

import glob
import librosa
fname_trn=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trn*.wav'):
    (fname_trn.append(filename))
fname_trn.sort()
# fname_trn

fname_trs=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trs*.wav'):
    (fname_trs.append(filename))
fname_trs.sort()
# fname_trs

fname_trx=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trx*.wav'):
    (fname_trx.append(filename))
fname_trx.sort()
# fname_trx[100]


In [9]:
import librosa

# sn, sr=librosa.load(fname_trn[0], sr=None)
# Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
# trn_arr=np.abs(Sn)
# print(trn_arr.shape)
# sx, sr=librosa.load(fname_trx[0], sr=None)
# Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
# trx_arr=np.abs(Sx)

# ss, sr=librosa.load(fname_trs[0], sr=None)
# Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
# trs_arr=np.abs(Ss)
count=0
total_train_s=[]
with open('train_n.txt', 'wb') as fn:
    for i in range(len(fname_trn)):
        sn, sr=librosa.load(fname_trn[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fn, mag_Sn, fmt='%.5f')
        fn.write(b'\n')
fn.close()            
#             np.savetxt(f, d[1], fmt='%.5f')

#     print(Sn.shape)

#     sx, sr=librosa.load(fname_trx[i], sr=None)
#     Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
#     mag_Sx=np.abs(Sx)
#     trx_arr=np.concatenate((trx_arr, mag_Sx), axis=1)
    
#     ss, sr=librosa.load(fname_trs[i], sr=None)
#     Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
#     mag_Ss=np.abs(Ss)
#     trs_arr=np.concatenate((trs_arr, mag_Ss), axis=1)
# print(Sn.shape)

# sn, sr=librosa.load(fname_trn[129], sr=None)
# Sn=librosa.stft(sn, n_fft=1024, hop_length=512)

# print(Sn.shape)

In [10]:
import librosa

with open('train_s.txt', 'wb') as fs:
    for i in range(len(fname_trs)):
        sn, sr=librosa.load(fname_trs[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fs, mag_Sn, fmt='%.5f')
        fs.write(b'\n')
fs.close()            


In [12]:
import librosa

with open('train_s_10.txt', 'wb') as fs:
    for i in range(10):
        sn, sr=librosa.load(fname_trs[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fs, mag_Sn, fmt='%.5f')
        fs.write(b'\n')
fs.close()            


In [25]:
f = open('train_s_10.txt', 'r')
x = f.read().split("\n")
f.close()


In [60]:
x=[]
with open('train_s_10.txt') as f:
    lines=f.readlines()
    print(len(lines))
    sentence_10=[]
    count = 0
    sentence=[]
    for line in lines:
        
        if count < 513:
            if count ==0:
                sentence=np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                print("sentence_1: ",sentence.shape)
                count+=1
            else:
#         print(line)
                myarray = np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
#                 print("myarray", myarray.shape)
                sentence=np.concatenate((sentence, myarray), axis=0)
                count+=1
        else:
            sentence_10.append(sentence) 
            count=0
            print("sentence_last", sentence.shape)
            sentence=[]
            sentence=np.fromstring(line, dtype=float, sep=' ')
            
            
            print("sentence_last", len(sentence_10))
#         sentence_10.append(sentence) 
        
#         print(myarray)
#         x.append(myarray)
# print(x.shape())

5140
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 1
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 2
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 3
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 4
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 5
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 6
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 7
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 8
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 9
sentence_1:  (1, 65)
sentence_last (513, 65)
sentence_last 10


In [45]:
len(sentence_10)

5140

In [11]:
import librosa

with open('train_x.txt', 'wb') as fs:
    for i in range(len(fname_trx)):
        sn, sr=librosa.load(fname_trx[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fs, mag_Sn, fmt='%.5f')
        fs.write(b'\n')
fs.close()            


In [18]:
import librosa

sn, sr=librosa.load(fname_trn[0], sr=None)
Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
trn_arr=np.abs(Sn)
print(trn_arr.shape)
sx, sr=librosa.load(fname_trx[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
trx_arr=np.abs(Sx)

ss, sr=librosa.load(fname_trs[0], sr=None)
Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
trs_arr=np.abs(Ss)
for i in range(1,1200):

    sn, sr=librosa.load(fname_trn[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    mag_Sn=np.abs(Sn)
    trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
#     print(Sn.shape)

    sx, sr=librosa.load(fname_trx[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
    mag_Sx=np.abs(Sx)
    trx_arr=np.concatenate((trx_arr, mag_Sx), axis=1)
    
    ss, sr=librosa.load(fname_trs[i], sr=None)
    Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
    mag_Ss=np.abs(Ss)
    trs_arr=np.concatenate((trs_arr, mag_Ss), axis=1)
# print(Sn.shape)

# sn, sr=librosa.load(fname_trn[129], sr=None)
# Sn=librosa.stft(sn, n_fft=1024, hop_length=512)

# print(Sn.shape)

(513, 65)


In [53]:
print(trn_arr.shape)
print(trx_arr.shape)
print(trs_arr.shape)

(513, 10, 11855)
(513, 10, 11855)
(513, 10, 11855)


In [20]:
DATA_train_M = 1*(trs_arr>trn_arr)
DATA_train_M.shape

(513, 118550)

In [40]:
trn_arr = trn_arr.reshape( (513,10,-1))
trx_arr = trx_arr.reshape( (513,10,-1))
trs_arr = trs_arr.reshape( (513,10,-1))
DATA_train_M = DATA_train_M.reshape( (513,10,-1))

trn_arr.shape, trx_arr.shape, trs_arr.shape, DATA_train_M.shape


((513, 10, 11855), (513, 10, 11855), (513, 10, 11855), (513, 10, 11855))

In [12]:
### validation Data

fname_val_n=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vn*.wav'):
    (fname_val_n.append(filename))
fname_val_n.sort()
# print(fname_val_n)

fname_val_s=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vs*.wav'):
    (fname_val_s.append(filename))
fname_val_s.sort()
# fname_trs

fname_val_x=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vx*.wav'):
    (fname_val_x.append(filename))
fname_val_x.sort()
# fname_trx[100]


In [14]:
import librosa

sn, sr=librosa.load(fname_val_n[0], sr=None)
Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
val_n_arr=np.abs(Sn)

# print(trn_arr.shape)
sx, sr=librosa.load(fname_val_x[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
val_x_arr=np.abs(Sx)

ss, sr=librosa.load(fname_val_s[0], sr=None)
Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
val_s_arr=np.abs(Ss)
for i in range(1,len(fname_val_n)):

    sn, sr=librosa.load(fname_val_n[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    mag_Sn=np.abs(Sn)
    val_n_arr=np.concatenate((val_n_arr, mag_Sn), axis=1)
#     print(Sn.shape)

    sx, sr=librosa.load(fname_val_x[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
    mag_Sx=np.abs(Sx)
    val_x_arr=np.concatenate((val_x_arr, mag_Sx), axis=1)
    
    ss, sr=librosa.load(fname_val_s[i], sr=None)
    Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
    mag_Ss=np.abs(Ss)
    val_s_arr=np.concatenate((val_s_arr, mag_Ss), axis=1)
# print(Sn.shape)

# sn, sr=librosa.load(fname_trn[129], sr=None)
# Sn=librosa.stft(sn, n_fft=1024, hop_length=512)

# print(Sn.shape)

In [15]:
DATA_val_M = 1*(val_s_arr>val_n_arr)
DATA_val_M.shape

(513, 118550)

In [41]:
val_n_arr = val_n_arr.reshape( (513,10,-1))
val_x_arr = val_x_arr.reshape( (513,10,-1))
val_s_arr = val_s_arr.reshape( (513,10,-1))
DATA_val_M = DATA_val_M.reshape( (513,10,-1))

val_n_arr.shape, val_x_arr.shape, val_s_arr.shape, DATA_val_M.shape


((513, 10, 11855), (513, 10, 11855), (513, 10, 11855), (513, 10, 11855))

In [7]:
### Test Data

fname_test=[]
for filename in glob.glob('/opt/e533/timit-homework/te/tex*.wav'):
    (fname_test.append(filename))
fname_test.sort()
# print(fname_val_n)

# fname_test_s=[]
# for filename in glob.glob('/opt/e533/timit-homework/te/vs*.wav'):
#     (fname_test_s.append(filename))
# fname_test_s.sort()
# # fname_trs

# fname_test_x=[]
# for filename in glob.glob('/opt/e533/timit-homework/te/vx*.wav'):
#     (fname_test_x.append(filename))
# fname_test_x.sort()
# # fname_trx[100]


['/opt/e533/timit-homework/te/tex0000.wav',
 '/opt/e533/timit-homework/te/tex0001.wav',
 '/opt/e533/timit-homework/te/tex0002.wav',
 '/opt/e533/timit-homework/te/tex0003.wav',
 '/opt/e533/timit-homework/te/tex0004.wav',
 '/opt/e533/timit-homework/te/tex0005.wav',
 '/opt/e533/timit-homework/te/tex0006.wav',
 '/opt/e533/timit-homework/te/tex0007.wav',
 '/opt/e533/timit-homework/te/tex0008.wav',
 '/opt/e533/timit-homework/te/tex0009.wav',
 '/opt/e533/timit-homework/te/tex0010.wav',
 '/opt/e533/timit-homework/te/tex0011.wav',
 '/opt/e533/timit-homework/te/tex0012.wav',
 '/opt/e533/timit-homework/te/tex0013.wav',
 '/opt/e533/timit-homework/te/tex0014.wav',
 '/opt/e533/timit-homework/te/tex0015.wav',
 '/opt/e533/timit-homework/te/tex0016.wav',
 '/opt/e533/timit-homework/te/tex0017.wav',
 '/opt/e533/timit-homework/te/tex0018.wav',
 '/opt/e533/timit-homework/te/tex0019.wav',
 '/opt/e533/timit-homework/te/tex0020.wav',
 '/opt/e533/timit-homework/te/tex0021.wav',
 '/opt/e533/timit-homework/te/te

In [8]:
sn, sr=librosa.load(fname_test[0], sr=None)
Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
test_arr=np.abs(Sn)

# print(trn_arr.shape)
# sx, sr=librosa.load(fname_val_x[0], sr=None)
# Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
# val_x_arr=np.abs(Sx)

# ss, sr=librosa.load(fname_val_s[0], sr=None)
# Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
# val_s_arr=np.abs(Ss)
for i in range(1,len(fname_test)):

    sn, sr=librosa.load(fname_test[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    mag_Sn=np.abs(Sn)
    test_arr=np.concatenate((test_arr, mag_Sn), axis=1)


In [42]:
test_arr.shape
test_arr = test_arr.reshape( (513,10,-1))
test_arr.shape

(513, 10, 4415)

In [43]:
np.save('DATA_train_s.npy', trs_arr)
np.save('DATA_train_n.npy', trn_arr)
np.save('DATA_train_x.npy', trx_arr)
np.save('DATA_train_M.npy', DATA_train_M)

np.save('DATA_val_s.npy', val_s_arr)
np.save('DATA_val_n.npy', val_n_arr)
np.save('DATA_val_x.npy', val_x_arr)
np.save('DATA_val_M.npy', DATA_val_M)

np.save('DATA_test_x.npy', test_arr)

In [44]:
DATA_train_s_f = np.load('DATA_train_s.npy')
DATA_train_n_f = np.load('DATA_train_n.npy')
DATA_train_x_f = np.load('DATA_train_x.npy')
DATA_train_M_f = np.load('DATA_train_M.npy')

DATA_val_s_f = np.load('DATA_val_s.npy')
DATA_val_n_f = np.load('DATA_val_n.npy')
DATA_val_x_f = np.load('DATA_val_x.npy')
DATA_val_M_f = np.load('DATA_val_M.npy')

DATA_test_x_f = np.load('DATA_test_x.npy')

In [52]:
DATA_val_M_f[:10, :3, :10]

array([[[0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 1, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 

In [45]:
tr_categorical_labels = to_categorical(DATA_train_M_f.T, num_classes=2)
print(tr_categorical_labels.shape)

v_categorical_labels = to_categorical(DATA_val_M_f.T, num_classes=2)
print(v_categorical_labels.shape)

(11855, 10, 513, 2)
(11855, 10, 513, 2)


In [54]:
Max_RNN = 10
# create the model

model = Sequential()

model.add(LSTM(Max_RNN, return_sequences=True, input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))

model.add(Dense(513, activation='sigmoid'))
          
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# model.fit(DATA_train_x_f.T, tr_categorical_labels, validation_data=(DATA_val_x_f.T,v_categorical_labels), shuffle=True, nb_epoch=100, batch_size=10)
# # Final evaluation of the model
# scores = model.evaluate(DATA_val_x_f.T, v_categorical_labels, verbose=0)



model.fit(DATA_train_x_f.T, DATA_train_M_f.T, validation_data=(DATA_val_x_f.T,DATA_val_M_f.T), shuffle=True, nb_epoch=25, batch_size=10)
# Final evaluation of the model
scores = model.evaluate(DATA_val_x_f.T, DATA_val_M_f.T, verbose=0)



print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10, 10)            20960     
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 10)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 10, 513)           5643      
Total params: 26,603
Trainable params: 26,603
Non-trainable params: 0
_________________________________________________________________
None


/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 11855 samples, validate on 11855 samples
Epoch 1/25
11855/11855 [==============================] - 20s 2ms/step - loss: 0.2133 - acc: 0.0017 - val_loss: 0.1920 - val_acc: 0.0018
Epoch 2/25
11855/11855 [==============================] - 20s 2ms/step - loss: 0.1944 - acc: 0.0026 - val_loss: 0.1836 - val_acc: 0.0022
Epoch 3/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1891 - acc: 0.0033 - val_loss: 0.1782 - val_acc: 0.0022
Epoch 4/25
11855/11855 [==============================] - 20s 2ms/step - loss: 0.1859 - acc: 0.0049 - val_loss: 0.1751 - val_acc: 0.0035
Epoch 5/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1838 - acc: 0.0076 - val_loss: 0.1735 - val_acc: 0.0071
Epoch 6/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1824 - acc: 0.0124 - val_loss: 0.1718 - val_acc: 0.0091
Epoch 7/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1812 - acc: 0.0162 - val_loss: 0.1712 - val_acc

In [55]:
import librosa

sx, sr=librosa.load(fname_trx[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
X_tr=Sx

for i in range(1,len(fname_trx)):

    sx, sr=librosa.load(fname_trx[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
#     mag_Sx=np.abs(Sx)
    X_tr=np.concatenate((X_tr, Sx), axis=1)
    


In [ ]:
import librosa

sx, sr=librosa.load(fname_val_x[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
X_val=Sx

for i in range(1,len(fname_val_x)):

    sx, sr=librosa.load(fname_val_x[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
#     mag_Sx=np.abs(Sx)
    X_val=np.concatenate((X_val, Sx), axis=1)
    


In [48]:
labels = model.predict(DATA_val_x_f.T)
labels.shape

(11855, 10, 513)

In [49]:
S_hat = (labels.T) * DATA_val_x_f

In [50]:
S_org = (DATA_val_s_f).flatten()
S_pred = S_hat.flatten()

In [51]:
acc = np.sum(S_org*S_org)/ np.sum((S_org-S_pred)*(S_org-S_pred))
print(acc)

10*np.log10(acc)

4.074747


6.101006865501404